In [1]:
# Read in dependencies
import pandas as pd
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from datetime import datetime

from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date

In [2]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [3]:
# Create a connection to the engine
conn = engine.connect()

In [5]:
Base = declarative_base()

class Measurements(Base):
    __tablename__ = 'measurements'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Date)
    prcp = Column(Float)  
    tobs = Column(Float)

In [6]:
Base.metadata.create_all(engine)

In [8]:
measurements_df = pd.read_csv("/Users/jr/Desktop/hawaii_measurements.csv")
measurements_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,NaN,73


In [9]:
# convert date column to datetime dtype
measurements_df['date'] = pd.to_datetime(measurements_df['date'], format = '%Y-%m-%d')

In [10]:
# Use Orient='station' to create a list of data to write
data_measurements = measurements_df.to_dict(orient='records')

In [11]:
data_measurements[0]

{'date': Timestamp('2010-01-01 00:00:00'),
 'prcp': 0.08,
 'station': 'USC00519397',
 'tobs': 65}

In [12]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [13]:
# Save the reference to the `stations` table as a variable called `station_table`
measurement_table = sqlalchemy.Table('measurements', metadata, autoload=True)

In [14]:
conn.execute(measurement_table.delete())

In [15]:
# Use `table.insert()` to insert the data into the table
conn.execute(measurement_table.insert(), data_measurements)

In [16]:
# Testing insters
conn.execute("select * from measurements limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-06', None, 73.0)]

In [17]:
class Stations(Base):
    __tablename__ = 'stations'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Float)  
    longitude = Column(Float)
    elevation = Column(Float)

In [18]:
Base.metadata.create_all(engine)

In [20]:
stations_df = pd.read_csv("/Users/jr/Desktop/clean_hawaii_stations.csv")
stations_df.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [21]:
# Use Orient='records' to create a list of data to write
station_data = stations_df.to_dict(orient='records')
station_data[0]

{'elevation': 3.0,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [22]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [23]:
# Save the reference to the `stations` table as a variable called `station_table`
station_table = sqlalchemy.Table('stations', metadata, autoload=True)

In [24]:
# Use `table.delete()` to remove any existing data.
conn.execute(station_table.delete())

In [25]:
# Use `table.insert()` to insert the data into the table
conn.execute(station_table.insert(), station_data)

In [26]:
# Testing the insert
conn.execute("select * from stations limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]